In [ ]:
from neuralforecast.models.mixmodel_rescale_deco_nomnk import MIXModel as nomnk
from neuralforecast.models.mixmodel_rescale_deco_nodsp import MIXModel as nodsp

import pandas as pd
from neuralforecast.core import NeuralForecast
from neuralforecast.models.mixmodel_rescale_deco import MIXModel
from neuralforecast.losses.pytorch import MAE, MSE
from neuralforecast.losses.numpy import mae, mse
from datasetsforecast.long_horizon2 import LongHorizon2, LongHorizon2Info

def exp(horizon,dataset,modelname):
  input_size = 192
  epochs = 300
  
  if(modelname =='nodsp'):
    model = nodsp(
          h = horizon,
          input_size = input_size,
          decomp_kernel_size = 25, 
          transformer_input_size = 192,
          n_blocks = [1, 1, 1],
          mlp_units = 3 * [[256, 256]],
          n_pool_kernel_size = [16, 8, 1],
          n_freq_downsample = [16, 8, 1],
          pooling_mode = "MaxPool1d",
          interpolation_mode = "linear",
          dropout_prob_theta=0.2,
          mlp_activation = "ReLU",
          
          encoder_layers = 3,
          n_heads = 4,
          hidden_size = 64,
          linear_hidden_size = 128,
          dropout = 0.2,
          fc_dropout = 0.2,
          head_dropout = 0.0,
          attn_dropout = 0.0,
          patch_len = 16,
          stride = 8,
          revin = True,
          revin_affine = False,
          revin_subtract_last = True,
          activation = "gelu",
          res_attention = True,
          batch_normalization = False,
          learn_pos_embed = True,
          loss=MAE(),
          valid_loss=MAE(),
          max_steps = epochs,
          learning_rate = 1e-4,
          num_lr_decays = -1,
          early_stop_patience_steps = 3,
          val_check_steps = 20,
          batch_size = 128,
          valid_batch_size = None,
          windows_batch_size = 1024, 
          inference_windows_batch_size = 1024,
          step_size = 2, 
          scaler_type = "identity",
          random_seed = 2023,
          num_workers_loader = 0, # 取决于 cpu 数量
          stat_exog_list=None,
          hist_exog_list=None,
          futr_exog_list=None,
          exclude_insample_y=False,
          drop_last_loader = False,
    )
  else:
    model = nomnk(
      h = horizon,
          input_size = input_size,
          decomp_kernel_size = 25, 
          transformer_input_size = 192,
          n_blocks = [1, 1, 1],
          mlp_units = 3 * [[256, 256]],
          n_pool_kernel_size = [16, 8, 1],
          n_freq_downsample = [16, 8, 1],
          pooling_mode = "MaxPool1d",
          interpolation_mode = "linear",
          dropout_prob_theta=0.2,
          mlp_activation = "ReLU",
          
          encoder_layers = 3,
          n_heads = 4,
          hidden_size = 64,
          linear_hidden_size = 128,
          dropout = 0.2,
          fc_dropout = 0.2,
          head_dropout = 0.0,
          attn_dropout = 0.0,
          patch_len = 16,
          stride = 8,
          revin = True,
          revin_affine = False,
          revin_subtract_last = True,
          activation = "gelu",
          res_attention = True,
          batch_normalization = False,
          learn_pos_embed = True,
          loss=MAE(),
          valid_loss=MAE(),
          max_steps = epochs,
          learning_rate = 1e-4,
          num_lr_decays = -1,
          early_stop_patience_steps = 3,
          val_check_steps = 20,
          batch_size = 128,
          valid_batch_size = None,
          windows_batch_size = 1024, 
          inference_windows_batch_size = 1024,
          step_size = 2, 
          scaler_type = "identity",
          random_seed = 2023,
          num_workers_loader = 0, # 取决于 cpu 数量
          stat_exog_list=None,
          hist_exog_list=None,
          futr_exog_list=None,
          exclude_insample_y=False,
          drop_last_loader = False,
    )
  # read data：
  Y_df = LongHorizon2.load(directory="./data/", group=dataset)
  Y_df["ds"] = pd.to_datetime(Y_df["ds"])
  n_time = LongHorizon2Info[dataset].n_time
  freq = LongHorizon2Info[dataset].freq
  val_size = LongHorizon2Info[dataset].val_size
  test_size = LongHorizon2Info[dataset].test_size
  Y_df.drop(columns=["index"], inplace=True)
  # fit & test model
  nf = NeuralForecast(models=[model], freq=freq)
  Y_hat_df = nf.cross_validation(df=Y_df, val_size=val_size, test_size=test_size,step_size=1, n_windows=None)
  # calculate matrics
  y_true = Y_hat_df.y.values
  y_hat = Y_hat_df["MIXModel"].values
  n_series = len(Y_hat_df.unique_id.unique())
  y_true = y_true.reshape(n_series, -1, horizon)
  y_hat = y_hat.reshape(n_series, -1, horizon)
  return mae(y_hat, y_true), mse(y_hat, y_true)

modelnames = ['nodsp','nomnk']
horizons = [48, 96, 192, 336, 720]
datasets = ['Exchange']

result = pd.DataFrame(columns=['model','dataset','horizon','mae','mse'])
for modelname in modelnames:
  for dataset in datasets:
    for horizon in horizons:
      _mae, _mse = exp(horizon,dataset,modelname)
      result = result._append({'model':modelname,'dataset':dataset,'horizon':horizon,'mae':_mae,'mse':_mse},ignore_index=True)
result.to_csv('./data/ablation_module.csv')

In [3]:
result

,model,dataset,horizon,mae,mse
0,nodsp,Exchange,48,0.114551,0.028147
1,nodsp,Exchange,96,0.162026,0.053750
2,nodsp,Exchange,192,0.239578,0.112282
3,nodsp,Exchange,336,0.331283,0.209334
4,nodsp,Exchange,720,0.556773,0.542195
5,nomnk,Exchange,48,0.114379,0.028489
6,nomnk,Exchange,96,0.162031,0.054900
7,nomnk,Exchange,192,0.248077,0.121696
8,nomnk,Exchange,336,0.351602,0.234414
9,nomnk,Exchange,720,0.575561,0.573570


In [ ]:
result.to_csv('./data/ablation_module_Exchange.csv')